In [2]:
import pandas as pd                    
import xml.etree.ElementTree as ET     

N_CVs = 1 
linhas = [] # uma linha para cada artigo lido

# Carrega CVs e cria um dataframe com os dados dos artigos lidos
for i in range(1,N_CVs+1):
    arquivo = 'curriculo(' + str(i) + ').xml'
    print("Lendo arquivo: ", arquivo)
    doc_tree_root = (ET.parse(arquivo)).getroot()
    for elem in doc_tree_root:                
        if elem.tag == 'PRODUCAO-BIBLIOGRAFICA':            
            trabalhos = elem.findall('./TRABALHOS-EM-EVENTOS/TRABALHO-EM-EVENTOS')
            for trabalho in trabalhos:
                dados_basicos = trabalho.find('DADOS-BASICOS-DO-TRABALHO').attrib
                detalhes = trabalho.find('DETALHAMENTO-DO-TRABALHO').attrib
                autores = trabalho.findall('AUTORES')
                if dados_basicos.get('NATUREZA') == 'COMPLETO':
                    print('\tArtigo encontrado: ' + dados_basicos.get('TITULO-DO-TRABALHO'))                    
                    dados = dict(dados_basicos)
                    dados.update(detalhes)
                    lst = []
                    lst_autores= {}
                    for autor in autores:                        
                        lst.append(autor.get('NOME-COMPLETO-DO-AUTOR'))                      
                    lst_autores['AUTORES'] = lst
                    dados.update(lst_autores)                    
                    linhas.append(dados)                                        
prodPPGC = pd.DataFrame(linhas) # df é o dataframe contendo todos os dados lidos

Lendo arquivo:  curriculo(1).xml
	Artigo encontrado: GaAs MEMS Design Using 0.2um HEMT MMIC Technology
	Artigo encontrado: A Low-Power Enable/Disable GaAs MESFET Differential Logic
	Artigo encontrado: Monolithic Micromachined Planar Spiral Transformer
	Artigo encontrado: Modeling of Micromachined Planar Spiral Inductors
	Artigo encontrado: Providing Technology Infrastructure for MEMS
	Artigo encontrado: GaAs MESFET SRAM Using a New High Speed Memory Cell
	Artigo encontrado: A Low-Power Differential Cross-Coupled FET Logic for GaAs Asynchronous Design
	Artigo encontrado: DCFL- and DPTL-Based Approaches to Self-Timed GaAs Circuits
	Artigo encontrado: Can Asynchronous Design Reduce Power Dissipation in GaAs ICs?
	Artigo encontrado: New High Speed GaAs Memory Cell
	Artigo encontrado: Logica Diferencial de Bajo Consumo para el DiseÃ±o de Circuitos Digitales AsÃ­ncronos en GaAs
	Artigo encontrado: FPGA Design Migration: Some Remarks
	Artigo encontrado: Matriz Gate Array CMOS AvanÃ§ada, Confi

In [46]:
institution = pd.DataFrame(columns=['institution_name'])
production = pd.DataFrame(columns=['event_name', 'event_qualis', 'event_type'])
time = pd.DataFrame(columns=['year'])
fact_Table = pd.DataFrame(columns=['time_id', 'researcher_id', 'event_id', 'institution_id', 'count'])

In [47]:
author_lst = list()
for auth_lst in prodPPGC['AUTORES']:
    for auth in auth_lst:
        if auth not in author_lst:
            author_lst.append([auth, '', 'PPGC'])
researcher = pd.DataFrame(author_lst, columns=['name', 'research_area', 'program'])

In [51]:
researcher

,name,research_area,program
0,Nabil Bennouri,,PPGC
1,Jean Michel Karam,,PPGC
2,Bernard Courtois,,PPGC
3,Renato Perez Ribas,,PPGC
4,Alvaro Bernal,,PPGC
5,Alain Guyot,,PPGC
6,Renato Perez Ribas,,PPGC
7,Jerome Lescot,,PPGC
8,Jean Louis Leclercq,,PPGC
9,Jean Michel Karam,,PPGC
